In [1]:
import sqlite3
import numpy as np
import openai
import torch
import transformers
from transformers import BertTokenizer, BertModel
from concurrent.futures import ThreadPoolExecutor
from transformers import AutoModel, AutoTokenizer
from concurrent.futures import ProcessPoolExecutor, as_completed
import chromadb
import llm
from transformers import pipeline
import langchain
from langchain.chains import RetrievalQA
from langchain.vectorstores import Chroma
from langchain.chat_models import ChatOpenAI
from langchain.document_loaders import PyPDFLoader
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.llms import HuggingFacePipeline
from langchain.embeddings import SentenceTransformerEmbeddings
from sentence_transformers import SentenceTransformer
from langchain.text_splitter import RecursiveCharacterTextSplitter
import torch
import transformers
from transformers import LlamaForCausalLM, LlamaTokenizer

/home/ec2-user/SageMaker/.conda/envs/cuda12.2/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
vectordb = Chroma(persist_directory="./chroma_db", embedding_function=embeddings)

In [4]:
model_dir = 'Llama-2-7b-chat-hf/'
model = LlamaForCausalLM.from_pretrained(model_dir, local_files_only = True)

Loading checkpoint shards: 100%|██████████| 2/2 [01:01<00:00, 30.60s/it]


In [5]:
tokenizer = LlamaTokenizer.from_pretrained(model_dir)

In [31]:
pipeline = transformers.pipeline(
    "text-generation", 
    model=model,
    tokenizer=tokenizer,
    torch_dtype=torch.float16,
    device_map="auto",
    temperature= 0.8, 
    min_new_tokens = 150
)

In [32]:
llm = HuggingFacePipeline(pipeline=pipeline)

In [33]:
retriever = vectordb.as_retriever()

qa = RetrievalQA.from_chain_type(
    llm=llm, 
    chain_type="stuff", 
    retriever=retriever, 
    verbose=True
)


In [34]:
def rag(qa, query):
    print(f"Query: {query}\n")
    result = qa.run(query)
    print("\nResult: ", result)

In [ ]:
instruction = "You must provide only one answer, and it must be based solely on the context provided."
query = "What year did Herbert Hoover become president"
rag(qa, query + instruction)